In [1]:
#Code to download all needed libraries - needs to be run every time

from __future__ import division #used in Python2.x so the division btw integers result in float, default in python3
from pyomo.environ import * #library for modeling and optimisation
import argparse
from pyomo.opt import SolverStatus, TerminationCondition #provide status of solvers
import pandas as pd #data manipulation library
import numpy as np 
import pickle
import random
import sqlite3
import copy
import random
import csv
import gc

import warnings

import os
warnings.filterwarnings('ignore')


INTEREST = 0.03 #interest rate


path =    #specify the path to the working folder

In [5]:
#Model to run through all clusters one by one. Cluster information should be prior added to the input file as new columns - one column per cluster. 
#Values in the cluster column should correspond to segment ID for that cluster
#The code will solve optimization for each cluster separately

#note: cluster is sometimes referred to as group in the code


data_opt = pd.read_parquet(path + "2025-03-18_cells_30_complete_hsi_clustered_2_2.parquet")
print("data loaded")

class optimization:
    def __init__(self, cluster_column, all_data):  
        self.cluster_column = cluster_column
        
        all_data = data_opt

        
        all_data = all_data.set_index([cluster_column, 'stand', 'schedule', 'period'])

        all_data = all_data.fillna(0)  #Fill missing values with 0
        all_data['period'] = all_data.index.get_level_values(3)  #Extract year from index


        self.cluster_column = cluster_column  #Store the cluster column name
        self.data_opt = all_data  
        self.combinations = 1
        self.all_data = self.data_opt

        #Define unique index values dynamically
        self.Index_values = self.all_data.drop(['period'], axis=1).reset_index().set_index([cluster_column, 'stand', 'schedule']).index.unique()
        self.Group_index_values = self.all_data.drop(['period'], axis=1).reset_index().set_index([cluster_column, 'stand']).index.unique()

        self.area = self.all_data.loc[slice(None), slice(None), 1, 1]['area']

        #Fill missing values in the all_data DataFrame
        self.all_data = self.all_data.fillna(0)

        #Create the optimization model
        self.createModel()

    def createModel(self):
       
        self.model1 = ConcreteModel()

        
        self.model1.stands = Set(initialize=list(set(self.all_data.index.get_level_values(1))))
        self.model1.group = Set(initialize=list(set(self.all_data.index.get_level_values(0))))
        self.model1.year = Set(initialize=list(set(self.all_data.index.get_level_values(3))))
        self.model1.regimes = Set(initialize=list(set(self.all_data.index.get_level_values(2))))
        self.model1.scen_index = Set(initialize=[i for i in range(0, self.combinations)])
        self.model1.Index_values = self.Index_values
        self.model1.Group_index_values = self.Group_index_values

        #Index for decision variable (ID_group, stand, schedule)
        def index_rule(model1):
            index = []
            for (g, s, b) in model1.Index_values:
                index.append((g, s, b))
            return index
        self.model1.index1 = Set(dimen=3, initialize=index_rule)
        
        #Index for decision variable (ID_group, stand)
        def group_index_rule(model1):
            index = []
            for (g, s) in model1.Group_index_values:
                index.append((g, s))
            return index
        self.model1.stands_groups_index = Set(dimen=2, initialize=group_index_rule)

        #Decision variable: X1 now indexed by (ID_group, schedule)
        self.model1.X1 = Var(self.model1.index1, within=Boolean, initialize=1)

        #Adding new NPV variable and constraint
        self.model1.NPV = Var(within=NonNegativeReals)

        def NPV_INVENTORY(model1):
            row_sum = sum(self.all_data.npv.loc[(g, s, b, 6)] * self.all_data.area.loc[(g, s, b, 6)] * self.model1.X1[(g,s,b)] 
                        for (g, s, b) in self.model1.index1)
            return self.model1.NPV == row_sum

        self.model1.NPV_INV = Constraint(rule=NPV_INVENTORY)
        print("NPV constraint")


        # Constraint to ensure only one schedule per ID_group
        def regime_rule(model1,g, s):
            row_sum = sum(model1.X1[(g, s, b)] for b in [x[2] for x in model1.index1 if (x[0],x[1]) == (g,s)])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands_groups_index, rule=regime_rule)

        #Add binary variable B[g, b] to indicate which schedule is selected for group g (whether schedule b is selected for group g)
        self.model1.B = Var(self.model1.group, self.model1.regimes, within=Boolean)

        def regime_rule_same_branch_for_group(model1, g, s, b):
            # Ensure that the selected branch for each stand matches the selected branch for the group
            return model1.X1[(g, s, b)] == model1.B[g, b]

        self.model1.regime_same_branch_constraint = Constraint(self.model1.index1, rule=regime_rule_same_branch_for_group)

        def group_branch_selection_rule(model1, g):
            #Ensure that exactly one schedule is selected for group g
            return sum(model1.B[g, b] for b in model1.regimes) == 1

        self.model1.group_branch_selection_constraint = Constraint(self.model1.group, rule=group_branch_selection_rule)
        print("regime constraint")
        

        self.model1.Inc_min= Var(within=NonNegativeReals) #variable to store minimum income over all periods 
        self.model1.Income = Var(self.model1.year, within=NonNegativeReals) #variables for periodic incomes

        def Income_periodic(model1, t):
            row_sum = sum(self.all_data.INC.loc[(g,s,r,t)]*self.all_data.area.loc[(g,s,r,t)]* self.model1.X1[(g,s,r)]  for (g,s,r) in self.model1.index1)
            return self.model1.Income[t] == row_sum

        self.model1.Income_period = Constraint(self.model1.year, rule=Income_periodic)


        self.model1.target_income = Param(default=400000, mutable=True) #adjust minimum income target here if needed
        #Create a slack variable for each period (nonnegative)
        self.model1.slack = Var(self.model1.year, within=NonNegativeReals)

        def income_target_rule(model1, t):
            # Income[t] plus slack must meet or exceed target_income.
            return model1.Income[t] + model1.slack[t] >= self.model1.target_income
        self.model1.income_target = Constraint(self.model1.year, rule=income_target_rule)

        self.model1.TotalSlack = Var(within=NonNegativeReals, initialize=0)
       
        def total_slack_constraint(model1):
            return model1.TotalSlack == sum(model1.slack[k] for k in model1.year)
        self.model1.TotalSlackConstraint = Constraint(rule=total_slack_constraint)
        print("Slack constraint")

        #CHSI constraints
        self.all_data['CHSI'] = 1- ((1-self.all_data.LSWP)*(1-self.all_data.TTWP)*(1-self.all_data.LTT)*(1-self.all_data.CAP)*(1-self.all_data.HAZ))

        self.model1.Landscape_HSI_Tot = Var(within=NonNegativeReals)

        #Single constraint that computes the total HSI over all periods, groups, and stands.
        def total_HSI_constraint(model1):
            total_HSI = sum(
                model1.X1[(g, s, r)] * self.all_data.CHSI.loc[(g, s, r, k)] * self.all_data.area.loc[(g, s, r, k)]
                for (g, s, r) in model1.index1
                for k in model1.year
            )
            return model1.Landscape_HSI_Tot == total_HSI

        self.model1.LAND_HSI_TOTAL = Constraint(rule=total_HSI_constraint)
        print("CHSI constraint")

        #Equations to compute standing end volume 
        self.model1.EndVolume = Var(within=NonNegativeReals)

        def End_volume_constraint(model1):
            row_sum = sum(
                (self.all_data.V_Pine_end.loc[(g,s, r, 6)] + 
                self.all_data.V_Spruce_end.loc[(g,s, r, 6)] + 
                self.all_data.V_Birch_end.loc[(g,s, r, 6)]) * 
                self.all_data.area.loc[(g,s, r, 6)] * self.model1.X1[(g,s, r)]
                for (g,s, r) in self.model1.index1
            )
            return self.model1.EndVolume == row_sum
        self.model1.EndInv = Constraint(rule=End_volume_constraint)
        print("End volume")


        #Adjust minimum and maximum values for each objective (from payoff tables)
        self.model1.NPV_min = Param(default=1375007.80, mutable=True)
        self.model1.NPV_max = Param(default=2371796.30, mutable=True)


        self.model1.TotalSlack_min = Param(default=0, mutable=True)
        self.model1.TotalSlack_max = Param(default=2400000.00, mutable=True)


        self.model1.HSI_min = Param(default=4.85, mutable=True)
        self.model1.HSI_max = Param(default=171.43, mutable=True)


        self.model1.EndVol_min = Param(default=297.30, mutable=True)
        self.model1.EndVol_max = Param(default=10641.58, mutable=True)

        #Adjust Target values for each objective

        self.model1.NPV_target = Param(default=2000000, mutable=True)
        self.model1.EndVol_target = Param(default=8000, mutable=True)
        self.model1.TotalSlack_target = Param(default=100000, mutable=True)
        self.model1.HSI_target = Param(default=150, mutable=True)


        #Building achievement-scalarizing function here
        self.model1.D = Var(within=Reals)

        def NPV_D_constraint(model1):
            row_sum = ((self.model1.NPV - self.model1.NPV_target)/(self.model1.NPV_max-self.model1.NPV_min))
            return self.model1.D <= row_sum 
        self.model1.NPV_D = Constraint(rule=NPV_D_constraint)

        def TotalSlack_D_constraint(model1):
            row_sum = ((self.model1.TotalSlack_target-self.model1.TotalSlack)/(self.model1.TotalSlack_max - self.model1.TotalSlack_min))
            return self.model1.D <= row_sum 
        self.model1.TotalSlack_D = Constraint(rule=TotalSlack_D_constraint)

        def HSI_D_constraint(model1):
             row_sum = ((self.model1.Landscape_HSI_Tot - self.model1.HSI_target)/(self.model1.HSI_max - self.model1.HSI_min))
             return self.model1.D <= row_sum 
        self.model1.HSI_D = Constraint(rule=HSI_D_constraint)

        def End_V_D_constraint(model1):
            row_sum = ((self.model1.EndVolume - self.model1.EndVol_target)/(self.model1.EndVol_max - self.model1.EndVol_min))
            return self.model1.D <= row_sum 
        self.model1.END_V_D = Constraint(rule=End_V_D_constraint)

        print("D constrints")

        #Objective function
        def outcome_rule(model1):

            return ((self.model1.D + 0.001*(((self.model1.NPV)/(self.model1.NPV_max-self.model1.NPV_min)) + 
                    ((self.model1.EndVolume)/(self.model1.EndVol_max - self.model1.EndVol_min)) - 
                    ((self.model1.TotalSlack)/(self.model1.TotalSlack_max - self.model1.TotalSlack_min)) +
                    ((self.model1.Landscape_HSI_Tot)/(self.model1.HSI_max - self.model1.HSI_min)) 
                    
                    ))*10000) 
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)


    def solve(self):
        # Specify the solver and solve the model
        opt = SolverFactory('cbc') 
        #opt.options['mipgap'] = mip_gap  # Set the relative MIP gap
        self.results = opt.solve(self.model1, tee=True)



# Extract decision data
def GET_DECISION_DATA(opt_instance):
    gr = []
    st = []
    reg = []
    vals = []
    for (g, s, b) in opt_instance.model1.index1:
        gr.append(g)
        st.append(s)
        reg.append(b)
        vals.append(opt_instance.model1.X1[(g, s, b)].value)
    data = {opt_instance.cluster_column: gr, "stand": st, "schedule": reg, "value": vals}
    df = pd.DataFrame(data)
    df = df.set_index([opt_instance.cluster_column, 'stand', 'schedule'])
    return df


# 🔹 Run the optimization for all clusters
#cluster_columns = [f"cluster_{i}" for i in [48]] 

cluster_columns = [col for col in data_opt.columns if col.startswith("cluster_")]



# Initialize summary results list
summary_results = []

merged_results = None  # Initialize as None for merging

for cluster_col in cluster_columns:
    print(f"\n🔹 Running optimization for {cluster_col}...")

    t = optimization(cluster_col, data_opt)  # Run optimization
    t.solve()

    #Extract values of interest
    objective_value = value(t.model1.OBJ)
    npv_value = t.model1.NPV.value
    total_slack = t.model1.TotalSlack.value
    end_volume = t.model1.EndVolume.value
    hsi_tot = t.model1.Landscape_HSI_Tot.value

    #Collect income and slack for each period
    income_values = {f"Income_Period_{period}": value(t.model1.Income[period]) for period in t.model1.year}
    slack_values = {f"Slack_Period_{period}": value(t.model1.slack[period]) for period in t.model1.year}

    #Append to summary results
    summary_results.append({
        "Cluster": cluster_col,
        "Objective": objective_value,
        "NPV": npv_value,
        "Total_Slack": total_slack,
        "End_Volume": end_volume,
        "Landscape_HSI_Tot": hsi_tot,
        **income_values,
        **slack_values
    })

    #Extract decision data and rename column
    dec = GET_DECISION_DATA(t)
    dec = dec[["value"]].rename(columns={"value": f"value_{cluster_col}"})

    #Convert summary results list to DataFrame and save
    summary_df = pd.DataFrame(summary_results)
    summary_df.to_csv(path+"2025-04-09_Output_target16.csv", index=False)
    print(f"\n✅ Summary results written  ")

    #Convert data_opt to match the index structure before merging
    data_opt_indexed = data_opt.set_index(["stand", "schedule", "period"])

    #Select only required columns: stand, schedule, period, Tr, Rg
    data_opt_selected = data_opt_indexed[["Tr", "Rg"]]

    #Merge with the decision results
    final_decision_results = dec.merge(data_opt_selected, left_index=True, right_index=True, how="left")

    #Save the decision results
    final_decision_results.to_csv(path+"2025-04-09_Output_decisions_target16.csv")
    print(f"\n✅ Decision results written ")
    del t
    del dec
    gc.collect()




data loaded

🔹 Running optimization for cluster_48...
NPV constraint
regime constraint
Slack constraint
CHSI constraint
End volume
D constrints

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'D:\NMBU\TEMP\tmpwsebqydx.cplex.log' open.
CPLEX> New value for mixed integer optimality gap tolerance: 0.001
CPLEX> Problem 'D:\NMBU\TEMP\tmpote9fizm.pyomo.lp' read.
Read time = 0.78 sec. (41.53 ticks)
CPLEX> Problem name         : D:\NMBU\TEMP\tmpote9fizm.pyomo.lp
Objective sense      : Maximize
Variables            :  292457  [Nneg: 16,  Free: 1,  Binary: 292440]
Objective nonzeros   :       5
Linear constraints   :  280271  [Less: 4,  Greater: 6,  E